Trước khi run pip, cài đặt các thư viện bằng câu lệnh sau
```
pip install dash panda plotly keras==3.3.3 tensorflow xgboost scikit-learn numpy MetaTrader5
pip install dash --upgrade
```

In [ ]:
# Cài đặt các thư viện cần thiết
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
from keras.models import load_model as keras_load_model
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
import MetaTrader5 as mt
from datetime import datetime

import numpy as np

# Cấu hình
model_names = ['LSTM', 'RNN', 'XGB']
model_fullnames = {"LSTM": "LSTM","RNN": "RNN", "XGB": "XGBoost"}

col_names = ['Close', 'ROC']

timeframe = mt.TIMEFRAME_H1

rows = 1500
predict_rows = 1

# Khởi tạo ứng dụng Dash
app = dash.Dash()
server = app.server

In [ ]:
# Hàm lấy dữ liệu từ MetaTrader
mt.initialize()

login = 84193298
password = "-8WyMrVz"
server = "MetaQuotes-Demo"

mt.login(login, password, server)

def get_data_FX(symbol, timeframe, rows):
    ohlc_data = pd.DataFrame(mt.copy_rates_from_pos(symbol, timeframe, 0, rows))
    ohlc_data['time'] = ohlc_data.apply(lambda x: datetime.fromtimestamp(x["time"]), axis=1)
    ohlc_data = ohlc_data.rename(columns={'time': 'Date', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'real_volume': 'Volume'})
    ohlc_data.set_index("Date")
    return ohlc_data



In [ ]:
def get_predict_data(symbol, timeframe, rows, predict_rows, model_names, col_names):
    dataframe = get_data_FX(symbol, timeframe, rows)

    # Sắp xếp theo ngày, lấy cột ngày làm index
    dataframe["Date"]=pd.to_datetime(dataframe.Date,format="%Y-%m-%d")
    dataframe.index=dataframe['Date'].values

    dataframe=dataframe.sort_index(ascending=True,axis=0)

    # Tạo dataframe mới chứa dữ liệu cần thiết
    temp_df=pd.DataFrame(index=range(0,len(dataframe)),columns=['Date','Close', 'ROC'])

    for i in range(0,len(dataframe)):
        temp_df.loc[i, "Date"]=dataframe['Date'].iloc[i]
        temp_df.loc[i, "Close"]=dataframe["Close"].iloc[i]

    # Tính chỉ số ROC
    temp_df.loc[0, "ROC"] = None
    for i in range(1, len(dataframe)):
        temp_df.loc[i, "ROC"] = (temp_df.loc[i, "Close"] - temp_df.loc[i-1, "Close"]) / temp_df.loc[i-1, "Close"]

    temp_df.index=temp_df['Date'].values
    temp_df.drop("Date",axis=1,inplace=True)

    # Thêm predict_rows dòng vào cuối dữ liệu
    timediff = min(temp_df.index[-1] - temp_df.index[-2], temp_df.index[-2] - temp_df.index[-3])

    for i in range(predict_rows):
        next_time = temp_df.index[-1] + timediff
        temp_df.loc[next_time] = [None for _ in range(len(temp_df.columns))]
        
    dataframe = temp_df

    # Lấy mẫu từ dữ liệu để đưa ra dự đoán giá cổ phiếu bằng mô hình LSTM, RNN, XGB

    for model_name in model_names:
        for col_name in col_names:
            # Load model đã train
            if model_name == 'LSTM' or model_name == 'RNN':
                model = keras_load_model(f"model/{model_name.lower()}_{col_name.lower()}_model.keras")
            elif model_name == 'XGB':
                model = xgb.Booster()
                model.load_model(f"model/xgboost_{col_name.lower()}_model.json")

                
            # Thêm cột dự đoán vào dataframe
            dataframe.insert(dataframe.shape[1], f'Predictions_{col_name}_{model_name}', np.nan)
            # Giá trị dự đoán cho ngày gần nhất là giá trị thực tế
            dataframe.loc[dataframe.index[-predict_rows-1], f'Predictions_{col_name}_{model_name}'] = dataframe.loc[dataframe.index[-predict_rows-1], col_name]

            # Lấy dữ liệu gồm 60 ngày gần nhất để dự đoán predict_rows ngày tiếp theo
            values=temp_df[len(temp_df)-(60+predict_rows):][col_name].values
            values=values.reshape(-1,1)

            scaler=MinMaxScaler(feature_range=(0,1))
            values=scaler.fit_transform(values)


            for i in range(predict_rows):
                X_test=[]
                X_test.append(values[i:i+60,0])
                X_test=np.array(X_test)

                if model_name == 'LSTM' or model_name == 'RNN':
                    X_test_3D=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
                    output=model.predict(X_test_3D)
                elif model_name == 'XGB':
                    X_test_DMatrix=xgb.DMatrix(X_test)
                    output=model.predict(X_test_DMatrix).reshape(-1, 1)

                values[60+i]=output[0][0]
                output=scaler.inverse_transform(output)

                # Giá trị dự đoán cho ngày tiếp theo
                dataframe.loc[dataframe.index[-predict_rows+i], f'Predictions_{col_name}_{model_name}'] = output[0][0]

    return dataframe

In [ ]:
app.layout = html.Div([

    html.H1("Forex Price Analysis Dashboard", style={"textAlign": "center"}),

    html.Div([
        html.Span("Select symbol: "),
        dcc.Dropdown(id='dropdown-stock',
                     
                    options=[
                        {'label': 'EURUSD', 'value': 'EURUSD'},
                        {'label': 'USDJPY', 'value': 'USDJPY'},
                        {'label': 'GBPUSD', 'value': 'GBPUSD'}
                        ],
                    multi=False,value='EURUSD',
                    style={"display": "inline-block", "width": "10em"}),
    ],
    style={"display": "flex", "justify-content": "center", "align-items": "center"}),

    dcc.Tabs(id="tabs", children=[
        dcc.Tab(label='Prediction closing price', children=[
            html.Div([
                html.H1("Prediction closing price",
                        style={'textAlign': 'center'}),

                dcc.Graph(id='prediction'),

            ]),
        ]),

        dcc.Tab(label='Prediction ROC', children=[
            html.Div([
                html.H1("Prediction ROC",
                        style={'textAlign': 'center'}),
                        
                dcc.Graph(id='prediction_ROC'),

            ]),
        ])
        
    ]),

    
    dcc.Interval(
        id='interval-component',
        interval=60*1000,
        n_intervals=0
    )
])

@app.callback(Output('prediction', 'figure'),
              [Input('dropdown-stock', 'value'), Input('interval-component', 'n_intervals')])
def update_graph(selected_stock, n_intervals):
    selected_model = ["LSTM", "RNN", "XGB"]
    df = get_predict_data(selected_stock, timeframe, rows, predict_rows, model_names, ['Close'])
    
    trace1 = []
    trace2 = []
    
    trace1.append(
        go.Scatter(
            x=df.index, 
            y=df["Close"], 
            mode='lines', 
            opacity=0.6, 
            name="Actual Close Price", 
            textposition='bottom center'
        )
    )

    for model in selected_model:
        trace2.append(
            go.Scatter(
                x=df.index,
				y=df["Predictions_Close_" + model],
                mode='lines',
                opacity=0.6,
                name=f'Predictions_{model}',
                textposition='bottom center'
            )
        )

    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]

    figure = {
        'data': data,
        'layout': go.Layout(
            colorway=["#FF0000", '#00FF00', '#00CC33', '#009966', '#006699', '#0033CC'],
            height=600,
            title=f"Close price for {', '.join(str(model_fullnames[i]) for i in selected_model)} Over Time",
            xaxis={"title":"Date",
                'rangeselector': {
                    'buttons': list([
                        {'count': 1, 'label': '1H','step': 'hour','stepmode': 'backward'},
                        {'count': 1, 'label': '1D','step': 'day','stepmode': 'backward'},
                        {'count': 7, 'label': '1W','step': 'day','stepmode': 'backward'},
                        {'count': 1, 'label': '1M','step': 'month','stepmode': 'backward'},
                        {'count': 6, 'label': '6M','step': 'month', 'stepmode': 'backward'},
                        {'step': 'all'}
                    ])},
                    'rangeslider': {'visible': True}, 'type': 'date'
                },
                yaxis={"title":f"Close {selected_stock}"})
    }
    return figure


@app.callback(Output('prediction_ROC', 'figure'),
              [Input('dropdown-stock', 'value'), Input('interval-component', 'n_intervals')])
def update_graph(selected_stock, n_intervals):
    selected_model = ["LSTM", "RNN", "XGB"]
    df = get_predict_data(selected_stock, timeframe, rows, predict_rows, model_names, ['ROC'])
    model_fullnames = {"LSTM": "LSTM","RNN": "RNN", "XGB": "XGBoost"}
    trace1 = []
    trace2 = []
    
    trace1.append(
        go.Scatter(
            x=df.index, 
            y=df["ROC"],
            mode='lines',
            opacity=0.6,
            name="Actual ROC", 
            textposition='bottom center'
        )
    )

    for model in selected_model:
        trace2.append(
            go.Scatter(
                x=df.index,
				y=df["Predictions_ROC_" + model],
                mode='lines',
                opacity=0.6,
                name=f'Predictions_{model}',
                textposition='bottom center',
            )
        )

    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]

    figure = {
        'data': data,
        'layout': go.Layout(
            colorway=["#FF0000", '#00FF00', '#00CC33', '#009966', '#006699', '#0033CC'],
            height=600,
            title=f"ROC for {', '.join(str(model_fullnames[i]) for i in selected_model)} Over Time",
            xaxis={"title":"Date",
                'rangeselector': {
                    'buttons': list([
                        {'count': 1, 'label': '1H','step': 'hour','stepmode': 'backward'},
                        {'count': 1, 'label': '1D','step': 'day','stepmode': 'backward'},
                        {'count': 7, 'label': '1W','step': 'day','stepmode': 'backward'},
                        {'count': 1, 'label': '1M','step': 'month','stepmode': 'backward'},
                        {'count': 6, 'label': '6M','step': 'month', 'stepmode': 'backward'},
                        {'step': 'all'}
                ])},
                'rangeslider': {'visible': True}, 'type': 'date'
            },
            yaxis={"title":f"ROC {selected_stock}"})
    }
    return figure

In [ ]:
if __name__=='__main__':
	app.run(debug=True)
